# RoundRobinGroupChat

In [ ]:
import os  
from dotenv import load_dotenv  
  
from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import RoundRobinGroupChat  
from autogen_agentchat.conditions import TextMessageTermination  
from autogen_core import CancellationToken  
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_agentchat.ui import Console
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

server_params = SseServerParams(  
    url=mcp_server_uri,  
    headers={"Content-Type": "application/json"},  
    timeout=30  
)  

# Fetch tools (async)  
tools = await mcp_server_tools(server_params)  
print(f"Tools: {tools}")
# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

In [ ]:
len(tools)  # Check the number of tools available

## エージェント定義

In [ ]:
# Set up the assistant agent  
agent = AssistantAgent(  
    name="ai_assistant",  
    model_client=model_client,  
    tools=tools,  
    system_message=(  
        "あなたは役立つアシスタントです。複数のツールを使用して情報を検索し、質問に回答することができます。"  
        "利用可能なツールを確認し、必要に応じて使用してください。ユーザーが不明点がある場合は、確認のための質問をすることもできます。"
    )
)  


In [ ]:
# 終了条件を設定します：エージェントが自身として応答した際に停止します。 
termination_condition = TextMessageTermination("ai_assistant")

In [ ]:

loop_agent = RoundRobinGroupChat(  
    [agent],  
    termination_condition=termination_condition,  
)  


In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成
    task = "ユーザーID:123 の出荷状況を確認してください。"

    await Console(loop_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成
    task = "2024年9月1日から2024年9月2日までの間の合計受注金額を教えてください。"

    await Console(loop_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成
    task = "ユーザー別のツイート数上位を取得し、上位3名を教えてください。"

    await Console(loop_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成
    task = "#ローゼたん のツイートって何件あるの？検索して"

    await Console(loop_agent.run_stream(task=task))

In [ ]:
with tracer.start_as_current_span("RoundRobinGroupChat") as rollspan: # ルートスパンを作成
    task = "時間帯別ツイートで一番多い時間帯はいつ？"

    await Console(loop_agent.run_stream(task=task))